In [1]:
import random
from datetime import datetime, timedelta

import firebase_admin
from firebase_admin import credentials, db

import pandas as pd
import matplotlib.pyplot as plt

DATABASE_URL = "https://sit-225-5p-default-rtdb.firebaseio.com/sensors/DHT22/-OXflDv6TiBBKXzik2Z9"
SERVICE_KEY = "firebase-key.json"
PATH = "sensors/HC-SR04"
N_SAMPLES = 120
INTERVAL_S = 5

cred = credentials.Certificate(SERVICE_KEY)
firebase_admin.initialize_app(cred, {"databaseURL": DATABASE_URL})

def generate_sample(ts):
    base = 150.0
    # simple drift pattern + noise; clamp to 5..400 cm
    idx = int((pd.to_datetime(ts).timestamp()))
    drift = 30.0 if (idx // 100) % 2 == 0 else -30.0
    noise = random.uniform(-8.0, 8.0)
    dist = max(5.0, min(400.0, base + drift + noise))
    return {
        "sensor_name": "HC-SR04",
        "timestamp": ts,
        "distance_cm": round(dist, 2),
    }

def upload_samples():
    ref = db.reference(PATH)
    start = datetime.utcnow().replace(microsecond=0)
    for i in range(N_SAMPLES):
        ts = (start + timedelta(seconds=i * INTERVAL_S)).isoformat() + "Z"
        ref.push(generate_sample(ts))
    print(f"Uploaded {N_SAMPLES} samples to /{PATH}")

def download_csv_and_plot():
    ref = db.reference(PATH)
    data = ref.get() or {}
    df = pd.DataFrame.from_dict(data, orient="index")
    if df.empty:
        print("No data found at", PATH)
        return
    df = df.sort_values("timestamp")
    csv_path = "sr04_firebase_data.csv"
    img_path = "sr04_firebase_plot.png"
    df.to_csv(csv_path, index=False)

    t = pd.to_datetime(df["timestamp"])
    plt.figure()
    plt.plot(t, df["distance_cm"], label="Distance (cm)")
    plt.xlabel("Time")
    plt.ylabel("Distance (cm)")
    plt.title("HC-SR04 Readings (from Firebase)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(img_path, bbox_inches="tight")
    plt.close()
    print(f"Saved CSV -> {csv_path}")
    print(f"Saved plot -> {img_path}")

if __name__ == "__main__":
    upload_samples()
    download_csv_and_plot()


/var/folders/z3/2l40y8_n1hl9vns1b7_vgr1r0000gn/T/ipykernel_21115/261588218.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start = datetime.utcnow().replace(microsecond=0)


Uploaded 120 samples to /sensors/HC-SR04
Saved CSV -> sr04_firebase_data.csv
Saved plot -> sr04_firebase_plot.png
